[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stefanlessmann/ESMT_IML/blob/main/notebooks/p6_xai.ipynb)

# BADS Exercise 9 on Explainable AI (XAI)
<hr>
So far, we have talked about model training, the evaluation of predictive accuracy, and, of course, different learning algorithms. In this notebook, we want to discuss how we can explain ML models. Many potent models are also  complex, and thus their decisions can be hard to follow for humans. This is referred to as the interpretability vs. accuracy trade-off. 

Why does interpretability matter at all? When we decide whether or not someone will get a discount, making a wrong prediction entails low cost. However, when we talk about medicine or banks giving credits, or refusing to do so, the cost of an erroneous classification becomes much higher. Predictive accuracy, the perspective we used to emphasize when discussing model quality, remains useful but is often insufficient. To build trust in a model, we need to demonstrate that the model recommendations (e.g., predictions) are consistent with domain knowledge. Being able to explain the model-estimated feature-to-target relationship is the most important step. Even for predictive accuracy, understanding the feature-target relationship, which will determine predictions, is useful if not crucial. To see this, recall that we evaluate predictive accuracy on a test set, which is only a sample from the population. Say we have a model that predicts the test data highly accurately. Are we comfortable with believing that future data beyond the test set (sample) will be predicted with the same high degree of accuracy? Maybe, but typically we would want additional evidence. Knowing the way in which a model translates feature values into predictions and knowing that this translation is sensible brings the additional amount of comfort. Let's stress this point with a counterexample. Say you build a model to predict health risks. You would not trust a model that predicts health conditions to improve with excessive consumption of alcohol, smoking, deprivation of sleep, junk food, etc. We know these factors (i.e., feature values) are unhealthy and a model predicting, e.g., life expectation in years, must reflect this domain knowledge in its forecasts. Otherwise, no matter what test set accuracy might show, we would reject the model for not being plausible. This shows why knowing the feature-to-target relationship is crucial. Beyond building trust, regulations might demand insight into models, for example in financial contexts, and thus rule out opaque models. In this notebook, we will look into different techniques that promise to explain the decisions of ML models and make the feature-to-target relationship interpretable.


## Preliminaries
As always, we start with importing some standard packages and loading our (credit) data. Further, we need some black-box models the predictions of which we aim at interpreting. The corresponding codes are well-known from previous sessions; so no need for explanations. 

In [6]:
# Import standard packages. 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Loading the HMEQ credit risk data set
data_url = 'https://raw.githubusercontent.com/Humboldt-WI/bads/master/data/hmeq.csv'
print('Fetching data...')
hmeq = pd.read_csv(data_url)  # standard pandas function to load tabular data in CSV format

# Convert a category with k different values into k-1 binary variables. 
X = pd.get_dummies(hmeq, dummy_na=True, drop_first=True)
X = X.dropna().reset_index(drop=True)  # drop all cases with one or more missing value

# Separate the data into a matrix of feature values and a target variable
y = X.pop('BAD')
print('Loaded HMEQ credit risk data set with dimension (rows x columns) {} x {}'.format(*X.shape))

# Data partitioning
from sklearn.model_selection import train_test_split
rnd_state = 888  # Select a random state to make results reproducible
ts_frac = 0.3  # 30% of the data as hold-out
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts_frac, random_state = rnd_state)  

# Make yourself familiar with these vectors
print('Shape of the data ', y_train.shape, X_train.shape, y_test.shape, X_test.shape)

Fetching data...
Loaded HMEQ credit risk data set with dimension (rows x columns) 3515 x 18
Shape of the data  (2460,) (2460, 18) (1055,) (1055, 18)


Next we train a Random Forest (RF) classifier. RF is a complex - *opaque* - ML algorithm that often yields good performance. Since the model involves many base models, it is not at all clear how feature values and predictions related to another. Thus, RF is a good example for an approach that requires post-hoc xAI methods to explain forecasts.

In [1]:
#Fit a RF classifier and estimate the AUC score on the test set:


## Global feature importance analysis
Given the interest in xAI, it is not surprising that `sklearn` provides functionality for explaining model predictions. Being part of the `sklearn` environment, these functions are particularly easy to use and it makes sense to start with them. As the title of the section suggests, `sklearn`  supports global explanations. 

### Permutation-based feature importance
Permutation-based feature importance is a learner-agnostic way to judge the relevance of features. It produces an ordinal feature ranking. To achieve this, the algorithm permutes one feature by shuffling its values across all observations. This means each observation will receive a new feature value. This permutation breaks the relationship to the target variable. In other words, a learner should no longer be able to use the information in the feature to predict the target. Permutation-based feature importance exploits this by comparing the predictive performance of a model before and after permuting a feature. The higher the increase of the prediction error due to the permutation, the more important the feature. Repeating the comparison of model performance before vs. after permutation for all features, we obtain a ranking of features. Examine this for the RF classifier.

In [2]:
# Import permutation-based feature importance from sklearn.inspection & apply the function to the fitted
# RF, pay attention to the score function and to the parameter n_repeats:


# Sort features in descending order based on the permutation importance scores:


# Generate a bar-plot of RF feature importance:



Before moving on to other forms of xAI. Let's briefly discuss another - nicer - way to visualize the results of permutation-based feature importance analysis. Specifically, we can exploit the fact that the function `permutation_importance` repeats the calculation by default. Repetition implies that we have multiple importance scores per feature. Therefore, your next task is to generate a box-plot to show the results.

In [3]:
# Box plot of RF feature importance



To better understand the values of the feature importance scores, you can check out the [`sklearn` documentation where the calculation is detailed](https://scikit-learn.org/stable/modules/permutation_importance.html).  

## Partial dependence analysis
A partial dependence plot (PDP) depicts the **marginal** effect of a feature on model predictions, and this complements permutation-based feature importance analysis. Remember that the latter is useful to understand on which features a model relies. Afterward, however, we still do not know whether higher/lower values of a feature lead to higher/lower predictions. For example, does the model-estimated default probability increase or decrease when the debt-to-income ratio increases? A PDP answers this question and is, therefore, a natural complement to permutation-based feature importance. A PDP plots the values of a focal feature (on the x-axis) against model predictions (on the y-axis) whilst accounting for the combined effect of all other features (hence marginal effect). This marginalization is basically achieved by examining the model prediction for each value of the focal variable while averaging the values for other variables. We refer to the lecture on interpretable machine learning for a more formal coverage of partial-dependence analysis. Here, we proceed with a demo and examine the partial dependency between the model-estimated probability of default and the feature DEBTINC.

In [4]:
# Partial dependence of DEBTINC: use the function PartialDependenceDisplay from sklearn.inspection



In terms of functionality, `sklearn` has certainly a lot more to offer then creating a univariate PDP. The official [function description](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.plot_partial_dependence.html) and [sklearn documentation](https://scikit-learn.org/stable/modules/partial_dependence.html#partial-dependence) offer a lot of additional insight and examples. To transit into the next section on local interpretability, let's sketch one such example, an extension of the PDP to display not the model behavior as a whole but the development of predictions for every individual data point, i.e., every single borrower in our credit risk prediction context. This extension is known as the ICE plot, and is also supported by `PartialDependenceDisplay` by setting the argument `kind` to `individual` as follows.

In [5]:
#Generate ICE plot:



## Local Interpretability 
Techniques discussed up until here aim at interpreting a model at large. Therefore, they belong to the family of global interpretation methods. The lecture also introduced local interpretability and the SHAP framework. The following parts demonstrate this approach. 

Before moving on, please make sure to have the library installed on your machine. If needed, you could install it now using 
```bash
!pip install shap
```

### Shapley Additive Explanations

SHAP (SHapley Additive exPlanations) is a local, model-agnostic explanation method. The concept of the fair payout in a coalition, that is, the Shapley value, was introduced in the scope of cooperative game theory. The Shapley value captures the contribution a player makes to the outcome. <br> *When is this useful?* For example, if a football team wins a game that includes a cash price and wants to distribute this between the players based on their contribution to the result. <br> *How can we use this technique for model explanation?* In a machine learning context, our players are the features and the outcome is the prediction. So, we want to calculate the contribution of each feature to the prediction. <br>  *How is this achieved?* In a nutshell, this is done by changing the data slightly, and observing how the output changes, and then estimating how high each feature's contribution is to the prediction. Using SHAP values for interpretable machine learning was popularized by [Lundberg & Lee (2017)](https://papers.nips.cc/paper/2017/hash/8a20a8621978632d76c43dfd28b67767-Abstract.html).

For this part of the demo notebook, we will use the [SHAP package](https://github.com/slundberg/shap) to apply this method. If you are interested in learning how to write this function from scratch, please follow [this link](
https://afiodorov.github.io/2019/05/20/shap-values-explained/) for a detailed explanation, or ask ChatGPT ;)

#### Calculate SHAP values
To calculate SHAP values, make use of the `shap.TreeExplainer`, which receives a prediction model (e.g., fitted sklearn model object) as input. More specifically, as the name states, this explainer accepts tree models as input and explains their prediction. It turns out that the computation of SHAP values is very costly. To cope with computational challenges, tailor-made ways to approximate the Shapley value for specific types of prediction models are needed. The `SHAP` library includes solutions for trees, neural networks, and a model agnostic approach, which is much slower than the other two (kindly refer to the [GitHub repositroy](https://github.com/slundberg/shap) for details). 

Whilst SHAP is a local explanation method, this package also facilitates calculating SHAP values for all observations and provides graphical summaries of the entire SHAP values distribution to allow for global interpretation. 

In [6]:
# Import shap 

# Explainer algorithm  for tree models

# Calculate SHAP values for all observations
# **** Note that this computation is costly and may take a while to run ****



Subset the array of shapley values with the index 1, in order to get the SHAP values for the probability predictions corresponding to class 1 (`shap_values_rf[1]`). This way, you will focus on the prediction of the class default=1 or, differently put, bad payers. 

### SHAP plots
The Shap framework provides a ton of visualizations to diagnose ML models. Some target local interpretability. Others aggregate the individual-level SHAP values to facilitate global interpretability. 


Start with the *waterfall* plot. It belongs to the functions targeting local interpretability. Therefore, you have to choose a specific observation for inspection. Feel free to explore several observations. You might want to take a look at the observation with index 852.

In [7]:
# Chose row/ observation and extract values


#### Waterfall plot

In [8]:
# Local explanation SHAP plot for RF model for a specific test point:


An alternative visualization serving the same purpose is the *force plot*



#### Force plot

In [9]:
#Generate a force plot for a specific test data point:


In [10]:
# Generate a force plot for the first 100 data points:


#### SHAP plots for global interpretability
The SHAP library also featuers plots that mimic the approach taken by the global explanation methods discussed in the previous section. For example, to shed light on global feature importance, you can plot the SHAP values of every feature for every data point. The plot, called `beeswarm` plot, sorts features by the sum of SHAP value magnitudes over all data points, which gives a measure of a feature's overall influence on predictions.  

In the following task, generate a beeswarm plot and a bar plot.

In [11]:
# SHAP-based feature importance analysis with beeswarm plot:


In [12]:
# SHAP-based feature importance analysis with barplot:


Vertical dispersion at a single value of DEBTINC represents interaction effects with other features. To help reveal these interactions you can color by another feature. If you pass the whole explanation tensor to the color argument of the scatter plot, it will pick the best feature to color by.

In [14]:
# SHAP-based dependence analysis



In [15]:
# Generate summary plot for single observation, RF model



In [17]:
# Repeat for the shap values generated by an XGB model:



To see how and how much each variable impacts the output, look at the global summary plot. 


In [16]:
# Summary plot for whole RF not single observation



In [18]:
# Repeat for XGB:

